# Deep Learning Traiv

This notebook contains code for training and testing of different deep learning architectures.

## Balance dataset 
Balance the dataset as some classes have only a few images. Code retrieved from: https://medium.com/analytics-vidhya/how-to-apply-data-augmentation-to-deal-with-unbalanced-datasets-in-20-lines-of-code-ada8521320c9

In [1]:
from tensorflow.keras.utils import Sequence
from imblearn.over_sampling import RandomOverSampler
from imblearn.keras import balanced_batch_generator

class BalancedDataGenerator(Sequence):
    """ImageDataGenerator + RandomOversampling"""
    def __init__(self, x, y, datagen, batch_size=32):
        self.datagen = datagen
        self.batch_size = batch_size
        self._shape = x.shape        
        datagen.fit(x)
        self.gen, self.steps_per_epoch = balanced_batch_generator(x.reshape(x.shape[0], -1), y, sampler=RandomOverSampler(), batch_size=self.batch_size, keep_sparse=True)

    def __len__(self):
        return self._shape[0] // self.batch_size

    def __getitem__(self, idx):
        x_batch, y_batch = self.gen.__next__()
        x_batch = x_batch.reshape(-1, *self._shape[1:])
        return self.datagen.flow(x_batch, y_batch, batch_size=self.batch_size).next()

## Store data in h5 file for Google Colab

Reading information from Google Drive is very slow and the performance benefits of using a GPU on Google Colab are not visible. For more info: https://medium.com/@oribarel/getting-the-most-out-of-your-google-colab-2b0585f82403

### Read all images into in memory array with labels

In [46]:
def get_class_mapping(classes):
    classes.sort()
    class_to_idx = {i: classes[i] for i in range(len(classes))}
    return class_to_idx

In [48]:
import h5py
from PIL import Image
import os
import numpy as np

ROOT = 'dataset/Trainingsset/'
IMG_SIZE = (256, 256)

images, labels = [], []

# loop over directory classes to get all images
classes = [x for x in os.listdir(ROOT) if "." not in x]
class_mappings = get_class_mapping(classes)
print(class_mappings)


for c, v in class_mappings.items():
    c_path = os.path.join(ROOT, v)
    c_images = [os.path.join(c_path, x) for x in os.listdir(c_path) if not x.startswith('.')]
    print("found {} images for {}".format(len(c_images), v))
    for image_path in c_images:
        img = Image.open(image_path)
        
        # skip all grayscale images
        if img.mode != 'RGB':
            continue
            
        img = img.resize(IMG_SIZE)
        images.append(np.asarray(img))
        labels.append(c)
        
print("stored {} images and {} labels in array".format(len(images), len(labels)))

{0: 'Beach', 1: 'Desert', 2: 'Forest', 3: 'Galaxy', 4: 'Glacier', 5: 'Jungle', 6: 'Mountains', 7: 'Ruins', 8: 'Waterfalls'}
found 4303 images for Beach
found 1313 images for Desert
found 4373 images for Forest
found 526 images for Galaxy
found 970 images for Glacier
found 564 images for Jungle
found 7028 images for Mountains
found 478 images for Ruins
found 1209 images for Waterfalls
stored 20759 images and 20759 labels in array


In [49]:
ROOT = 'dataset/Testset/'
IMG_SIZE = (256, 256)

images_test, labels_test = [], []

for c, v in class_mappings.items():
    c_path = os.path.join(ROOT, v)
    c_images = [os.path.join(c_path, x) for x in os.listdir(c_path) if not x.startswith('.')]
    print("found {} images for {}".format(len(c_images), c))
    for image_path in c_images:
        img = Image.open(image_path)
        if img.mode == 'L':
            continue
            
        img = img.resize(IMG_SIZE)
        images_test.append(np.asarray(img))
        labels_test.append(c)
        
print("stored {} images and {} labels in array".format(len(images_test), len(labels_test)))

found 28 images for 0
found 27 images for 1
found 23 images for 2
found 26 images for 3
found 28 images for 4
found 21 images for 5
found 29 images for 6
found 28 images for 7
found 26 images for 8
stored 236 images and 236 labels in array


### Store images and labels in h5 file

In [57]:
fileName = 'data.h5'

with h5py.File(fileName, "w") as out:
    out.create_dataset("X_train", np.shape(images), dtype='u1', data=np.asarray(images))
    out.create_dataset("Y_train", np.shape(labels), dtype='u1',  data=np.asarray(labels))    
    out.create_dataset("X_test", np.shape(images_test), dtype='u1', data=np.asarray(images_test))
    out.create_dataset("Y_test", np.shape(labels_test), dtype='u1', data=np.asarray(labels_test))

## Data preprocessing and data augmentation

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32

train_data_dir = 'dataset/Trainingsset/'
img_size = (224,224)

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
        #validation_split=0.2)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = test_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 20764 images belonging to 9 classes.
Found 0 images belonging to 9 classes.


In [11]:
# get class weights
counter = Counter(train_generator.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}        
class_weights

{0: 1.6332791075993494,
 1: 5.352627570449353,
 2: 1.607134690144066,
 3: 13.361216730038024,
 4: 7.245360824742268,
 5: 12.460992907801419,
 6: 1.0,
 7: 14.702928870292887,
 8: 5.813068651778329}

In [6]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from collections import Counter

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(9, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# get class weights
counter = Counter(train_generator.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     


# train the model on the new data for a few epochs
step_size_train=train_generator.n//train_generator.batch_size
model.fit(
        train_generator,
        steps_per_epoch=step_size_train,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=validation_generator.n // batch_size,
        class_weight=class_weights)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(
        train_generator,
        steps_per_epoch=step_size_train,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=validation_generator.n // batch_size,
        class_weight=class_weights)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 519 steps, validate for 129 steps
Epoch 1/5
 64/519 [==>...........................] - ETA: 8:06 - loss: 8.1889 - accuracy: 0.1758

/Users/manuel/opt/anaconda3/envs/data-science/lib/python3.7/site-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


519/519 [==============================] - 622s 1s/step - loss: 5.3725 - accuracy: 0.3058 - val_loss: 4.5035 - val_accuracy: 0.5942
Epoch 2/5
519/519 [==============================] - 616s 1s/step - loss: 4.2141 - accuracy: 0.4001 - val_loss: 5.6441 - val_accuracy: 0.4542
Epoch 3/5
519/519 [==============================] - 599s 1s/step - loss: 4.0138 - accuracy: 0.4137 - val_loss: 4.7662 - val_accuracy: 0.5397
Epoch 4/5
519/519 [==============================] - 563s 1s/step - loss: 3.8453 - accuracy: 0.4300 - val_loss: 6.1935 - val_accuracy: 0.5579
Epoch 5/5
519/519 [==============================] - 562s 1s/step - loss: 3.8264 - accuracy: 0.4347 - val_loss: 5.3864 - val_accuracy: 0.4608
0 input_3
1 conv2d_188
2 batch_normalization_188
3 activation_188
4 conv2d_189
5 batch_normalization_189
6 activation_189
7 conv2d_190
8 batch_normalization_190
9 activation_190
10 max_pooling2d_8
11 conv2d_191
12 batch_normalization_191
13 activation_191
14 conv2d_192
15 batch_normalization_192
16 

/Users/manuel/opt/anaconda3/envs/data-science/lib/python3.7/site-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


519/519 [==============================] - 678s 1s/step - loss: 3.2937 - accuracy: 0.4644 - val_loss: 4.9145 - val_accuracy: 0.5952
Epoch 2/5
519/519 [==============================] - 721s 1s/step - loss: 2.9682 - accuracy: 0.5021 - val_loss: 4.4926 - val_accuracy: 0.6090
Epoch 3/5
519/519 [==============================] - 727s 1s/step - loss: 2.7669 - accuracy: 0.5235 - val_loss: 4.5219 - val_accuracy: 0.6102
Epoch 4/5
519/519 [==============================] - 727s 1s/step - loss: 2.6189 - accuracy: 0.5318 - val_loss: 4.6623 - val_accuracy: 0.5906
Epoch 5/5
519/519 [==============================] - 789s 2s/step - loss: 2.5408 - accuracy: 0.5459 - val_loss: 4.5498 - val_accuracy: 0.5911


In [8]:
len(train_generator.classes)

16616

In [ ]:
x, y = ... # load your data
datagen = ImageDataGenerator()
balanced_gen = BalancedDataGenerator(x, y, datagen, batch_size=32)
steps_per_epoch = balanced_gen.steps_per_epoch
model = ... # define your model
model.compile(...) # define your compile parameters
model.fit_generator(balanced_gen, steps_per_epoch, ...)